In [2]:
import requests
import zipfile
import os
import io

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Download the dataset
print("Downloading dataset...")
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
response = requests.get(url)
print("Download complete!")

# Extract the dataset
print("Extracting dataset...")
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall(".")
print("Extraction complete!")

Download complete!
Extracting dataset...
Extraction complete!


In [4]:
# Run this cell to install the required packages
!pip install torch torchvision torchaudio

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/111.0 MB 8.3 MB/s eta 0:00:14
   - -------------------------------------- 3.1/111.0 MB 10.2 MB/s eta 0:00:11
   - -------------------------------------- 4.2/111.0 MB 10.1 MB/s eta 0:00:11
   - -------------------------------------- 5.5/111.0 MB 7.6 MB/s eta 0:00:14
   -- ------------------------------------- 7.6/111.0 MB 8.0 MB/s eta 0:00:13
   --- ------------------------------------ 9.4/111.0 MB 8.2 MB/s eta 0:00:13
   ---- ----------------------------------- 11.3/111.0 MB 8.2 MB/s eta 0:00:13
   ---- ----------------------------------- 13.1/111.0 MB 8.2 MB/s eta 0:00:12
   ----- ---------------------------------- 14.9/111.0 MB 8.2 MB/s eta 0:00:12
   ------ ------------------------

In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check the dataset structure
print("Training data:")
print(os.listdir("data/train"))
print("\nTest data:")
print(os.listdir("data/test"))

Using device: cpu
Training data:
['curly', 'straight']

Test data:
['curly', 'straight']


In [3]:
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        
        # Convolutional layer with 32 filters, kernel size 3x3, and ReLU activation
        self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.relu = nn.ReLU()
        
        # Max pooling layer with pool size 2x2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Calculate the size after convolution and pooling
        # Input: (3, 200, 200)
        # After Conv2d(3, 32, 3x3): (32, 198, 198)
        # After MaxPool2d(2x2): (32, 99, 99)
        self.flatten_size = 32 * 99 * 99
        
        # Fully connected layers
        self.fc1 = nn.Linear(self.flatten_size, 64)
        self.fc2 = nn.Linear(64, 1)
        
        # No need to define sigmoid here as we'll use BCEWithLogitsLoss
        # which combines sigmoid and BCE in one function
        
    def forward(self, x):
        # Convolutional layer
        x = self.conv(x)
        x = self.relu(x)
        
        # Max pooling
        x = self.pool(x)
        
        # Flatten
        x = x.view(-1, self.flatten_size)
        
        # Fully connected layers
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# Create the model
model = HairClassifier().to(device)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Print model summary
print(model)

HairClassifier(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [4]:
criterion = nn.BCEWithLogitsLoss()

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [7]:
# Define transformations
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# Load datasets
train_dataset = datasets.ImageFolder(root='data/train', transform=train_transforms)
test_dataset = datasets.ImageFolder(root='data/test', transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

# Print class mappings
print(f"Class mappings: {train_dataset.class_to_idx}")

Class mappings: {'curly': 0, 'straight': 1}


In [8]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015


In [9]:
import numpy as np
# Calculate median of training accuracy
median_train_acc = np.median(history['acc'])
print(f"Median training accuracy: {median_train_acc:.4f}")

Median training accuracy: 0.8175


In [10]:
std_train_loss = np.std(history['loss'])
print(f"Standard deviation of training loss: {std_train_loss:.4f}")

Standard deviation of training loss: 0.1590


In [11]:
# Define transformations with augmentations
train_transforms_aug = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# Load dataset with augmentations
train_dataset_aug = datasets.ImageFolder(root='data/train', transform=train_transforms_aug)

# Create data loader with augmentations
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)

In [12]:
num_epochs = 10
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.7357, Acc: 0.5975, Val Loss: 0.6002, Val Acc: 0.6567
Epoch 2/10, Loss: 0.5931, Acc: 0.6325, Val Loss: 0.5686, Val Acc: 0.7214
Epoch 3/10, Loss: 0.5810, Acc: 0.6625, Val Loss: 0.5617, Val Acc: 0.7214
Epoch 4/10, Loss: 0.5631, Acc: 0.7100, Val Loss: 0.6174, Val Acc: 0.6617
Epoch 5/10, Loss: 0.5150, Acc: 0.7300, Val Loss: 0.5420, Val Acc: 0.7114
Epoch 6/10, Loss: 0.5132, Acc: 0.7325, Val Loss: 0.5480, Val Acc: 0.7264
Epoch 7/10, Loss: 0.5076, Acc: 0.7275, Val Loss: 0.4963, Val Acc: 0.7313
Epoch 8/10, Loss: 0.5164, Acc: 0.7325, Val Loss: 0.5050, Val Acc: 0.7562
Epoch 9/10, Loss: 0.5135, Acc: 0.7388, Val Loss: 0.5274, Val Acc: 0.7164
Epoch 10/10, Loss: 0.5027, Acc: 0.7488, Val Loss: 0.4982, Val Acc: 0.7463


In [13]:
# Calculate mean of test loss with augmentations
mean_val_loss_aug = np.mean(history_aug['val_loss'])
print(f"Mean test loss with augmentations: {mean_val_loss_aug:.4f}")

Mean test loss with augmentations: 0.5465


In [14]:
avg_val_acc_last_5 = np.mean(history_aug['val_acc'][5:])
print(f"Average test accuracy for last 5 epochs with augmentations: {avg_val_acc_last_5:.4f}")

Average test accuracy for last 5 epochs with augmentations: 0.7353
